In [19]:
with open("data/unico.txt", "r") as file:
    unico=file.read()
with open("data/separadas.csv", "r") as file:
    separadas=file.read()

In [73]:
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

In [74]:
tokenizer=Tokenizer(char_level=True)
tokenizer.fit_on_texts(unico)

In [75]:
tokenizer.texts_to_sequences(["hola"])

[[19, 4, 11, 2]]

In [76]:
tokenizer.word_index


{' ': 1,
 'a': 2,
 'e': 3,
 'o': 4,
 'r': 5,
 'i': 6,
 's': 7,
 'n': 8,
 't': 9,
 'c': 10,
 'l': 11,
 'd': 12,
 'm': 13,
 'u': 14,
 'p': 15,
 'v': 16,
 'b': 17,
 'g': 18,
 'h': 19,
 'q': 20,
 'j': 21,
 'f': 22,
 'y': 23,
 'í': 24,
 'z': 25,
 'ó': 26,
 'é': 27,
 'ñ': 28,
 'á': 29,
 'k': 30,
 'x': 31,
 'ú': 32,
 'w': 33,
 '…': 34,
 '¡': 35,
 'ü': 36,
 '’': 37,
 '¿': 38,
 '´': 39,
 'ã': 40,
 '\xad': 41,
 '³': 42,
 '±': 43,
 'â': 44,
 'ç': 45,
 'ê': 46,
 '”': 47,
 'ˆ': 48,
 'à': 49,
 '“': 50,
 'š': 51,
 '—': 52,
 'ý': 53,
 '¯': 54,
 '‘': 55,
 'ò': 56,
 '–': 57,
 '́': 58,
 '©': 59,
 'е': 60,
 'č': 61,
 'ć': 62,
 '•': 63}

In [77]:
max_id=len(tokenizer.word_index)
dataset_size=tokenizer.document_count
print("Number of distinct characters",max_id)
print("Total number of characters",dataset_size)

Number of distinct characters 63
Total number of characters 6519408


In [78]:
[encoded]=np.array(tokenizer.texts_to_sequences([unico]))-1
encoded.shape

(6519408,)

In [79]:
from tensorflow.data import Dataset
from tensorflow import one_hot
train_size=dataset_size * 90 // 100
print("train size is", train_size)
dataset=Dataset.from_tensor_slices(encoded[:train_size])

train size is 5867467


In [80]:
print(dataset.element_spec)

for element in dataset:
    print(element)
    break

TensorSpec(shape=(), dtype=tf.int64, name=None)
tf.Tensor(9, shape=(), dtype=int64)


In [81]:
n_steps=100 #Muy corto no capto contexto, muy largo se pierde, hay que revisarlo
window_length=n_steps + 1
dataset=dataset.window(window_length,shift=1, drop_remainder=True)

In [82]:
print(dataset)

<WindowDataset shapes: DatasetSpec(TensorSpec(shape=(), dtype=tf.int64, name=None), TensorShape([])), types: DatasetSpec(TensorSpec(shape=(), dtype=tf.int64, name=None), TensorShape([]))>


In [83]:
for element in dataset:
    print(element)
    break

<_VariantDataset shapes: (), types: tf.int64>


In [84]:
dataset=dataset.flat_map(lambda window: window.batch(window_length))

In [85]:
dataset

<FlatMapDataset shapes: (None,), types: tf.int64>

In [86]:
batch_size =32
dataset=dataset.shuffle(500).batch(batch_size) #Depende mi memoria ram esto
dataset=dataset.map(lambda windows: (windows[:,:-1],windows[:,1:]))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [87]:
for element in dataset:
    print(element)
    break

2022-03-04 14:36:52.882709: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-04 14:36:52.918626: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2304010000 Hz


(<tf.Tensor: shape=(32, 100), dtype=int64, numpy=
array([[ 3, 20,  1, ...,  0, 12,  1],
       [ 5,  7, 11, ...,  0, 19, 13],
       [ 2,  0,  1, ...,  2,  7,  0],
       ...,
       [ 0, 10,  3, ..., 16, 10,  2],
       [ 0,  2, 18, ...,  0, 15,  3],
       [10,  2,  0, ...,  0, 15,  1]])>, <tf.Tensor: shape=(32, 100), dtype=int64, numpy=
array([[20,  1, 10, ..., 12,  1,  6],
       [ 7, 11,  0, ..., 19, 13,  2],
       [ 0,  1,  6, ...,  7,  0,  6],
       ...,
       [10,  3,  3, ..., 10,  2, 12],
       [ 2, 18,  0, ..., 15,  3, 10],
       [ 2,  0, 10, ..., 15,  1, 22]])>)


In [89]:
dataset=dataset.map(lambda X_batch,Y_batch: (one_hot(X_batch,depth=max_id),Y_batch))

In [90]:
dataset

<MapDataset shapes: ((None, None, 63), (None, None)), types: (tf.float32, tf.int64)>

In [91]:
dataset=dataset.prefetch(1)

In [92]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 63) (32, 100)


## Train

In [104]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU
from keras.layers import TimeDistributed, Dense
import keras

In [100]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU
from keras.layers import TimeDistributed, Dense
model =Sequential([
    GRU(128, return_sequences=True, input_shape=[None, max_id],
                       #dropout=0.2, recurrent_dropout=0.2),
                        dropout=0.2),
    GRU(128, return_sequences=True,
                   #dropout=0.2, recurrent_dropout=0.2),
                    dropout=0.2),
    TimeDistributed(Dense(max_id,activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history=model.fit(dataset,epochs=10)

NotImplementedError: Cannot convert a symbolic Tensor (gru_8/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [105]:
model=keras.models.Sequential()
model.add(keras.layers.GRU(128,return_sequences=True,input_shape=[None,max_id]))
model.add(keras.layers.GRU(128,return_sequences=True))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax')))


NotImplementedError: Cannot convert a symbolic Tensor (gru_10/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported